## Clustering Algorithms

Source: https://scikit-learn.org/stable/modules/clustering.html

### Sci-kit Learn on Input Data

**Input Data**

One important thing to note is that the algorithms implemented in this module can take different kinds of matrix as input. All the methods accept standard data matrices of shape (n_samples, n_features). These can be obtained from the classes in the sklearn.feature_extraction module. For AffinityPropagation, SpectralClustering and DBSCAN one can also input similarity matrices of shape (n_samples, n_samples). These can be obtained from the functions in the sklearn.metrics.pairwise module.

### K-Means

**Parameters** - Number of Clusters

**Scalability** - Very Large n_samples, medium n_clusters with MiniBatch code

**Usecase** - General-purpose, even cluster size, flat geometry, not too many clusters

**Metric Used** - Distances between points

The K-means algorithm clusters data by trying to separate samples in n groups of equal variance, minimizing a criterion known as the inertia or within-cluster sum of squares. It requires the number of clusters to be specified and scales well to large number of samples. It is a general purpose clustering algorithm.

The algorithm divides a set of **N** samples **X** into **K** disjoint clusters **C** each described by the mean $\mu_j$ of the samples in the cluster. The means are commonly called the cluster "centroids". Centroids are not generally points in **X**, although the points that are centroids are in the same space.

The metric minimized is the **inertia** or **within-cluster sum-of-squares criterion**:

<p style="text-align: center;"> $\underset{i=0}{\sum}^n \underset{\mu_j \in C}{min}(\|x_i - \mu_j \|^2)$
</p>

Inertia can be thought of as a measure of how internally coherent clusters are. It suffers from a few problems.

* Inertia makes the assumption that clusters are convex and isotropic, which is not always the case. It responds poorly to elongated clusters, or manifolds with irregular shapes.
* Inertia is not a normalized metric: we just know that lower values are better and zero is optimal. But in very high-dimensional spaces, Euclidean distances tend to become inflated (curse of dimensionality). Using a dimensionality reduction algorithm such as PCA prior to k-means clustering can alleviate this problem and speed up computations.

There are 3 steps to this algorithm.

1. Choose initial centroids.

2. Assign each sample to each nearest centroid.

3. Create new centroids by taking the mean value of all of the samples assigned to each previous centroid.

The difference between the old and new centroids are computed and the algorithm repeats the last two steps until this value is less than some treshold.

K-Means is equivalent to the expecation-maximization algorithm witha  small, all-equal, disagonal covariance matrix.

Given enough time, K-means will always converge, however this may be to a local minimum. This is highly dependent on the initialization of the centroids. 

The algorithm also supports sample weights. This allows us to assign more weight to some samples when computing cluster centers and values of inertia.

#### Mini Batch K-Means

The MiniBatchKMeans is a variant of the KMeans algorithm which uses mini-batches to reduce computation time, while still attempting to optimize the same objective function. Mini-batches are subsets of the input data, randomly sampled in each training iteration. These mini-batches drastically reduce the amount of computation required to converge to a local solution. In contrast to other algorithms that reduce the convergence time of k-means, mini-batch k-means produces results that are generally only slightly worse than the standard algorithm.

The algorithm iterates between two major steps, similar to vanilla K-Means.

1. In the first step, **b** samples are drawn randomly from the dataset, to form a minibatch. These are then assigned to the nearest centroid. 

2. Centroids are updated. In contrast to K-Means, this is done on a per-sample basis. For each sample in the mini-batch, the assigned centroid is updated by taking the streaming average of the samplea nd all previous samples assigned to that centroid. This has the effect of decreasing the rate of change for a centroid over time.

These steps are performed until convergence or a predetermined number of iterations is reached.

MiniBatchKMeans converges faster than KMeans, but the quality of the results is reduced. In practice this difference in quality can be quite small.

#### Initialization

There are two supported initialization methods in the scikit-learn library: K-Means++ and a random initialization. 

K-Means++ - An algorithm for choosing the initial values (or seeds/centroids). The algorithm is as follows.

1. Choose one center uniform at random among the data points.
2. For each data point x compute its distance from the nearest, previously chosen centroid.
3. Select the next centroid from the data points such that the probability of choosing a point as a centroid is directly proportional to its distance from the nearest, previously chosen centroid. (i.e the point having the maximum distance from the nearest centroid is most likely to be selected next as a centroid)
4. Repeat steps 2 and 3 until k centroids have been sampled.

Sources

https://en.wikipedia.org/wiki/K-means%2B%2B

https://www.real-statistics.com/multivariate-statistics/cluster-analysis/initializing-clusters-k-means/

https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_plusplus.html

https://www.geeksforgeeks.org/ml-k-means-algorithm/

Random - Choose n_clusters (k) observations (rows) at random from data for the initial centroids.

#### Final Notes

The average complexity is given by O(k n T), where n is the number of samples and T is the number of iterations. The worst case is given by O($n^{(k+2)/p}$) with n = n_samples, p = n_features.

In practice, the K-Means algorithm is very fast (one of the fastest clustering algorithms available), but it falls in local minima. This is why it can be useful to restart it several times.

**tol : float, default=$1x10^{-4}$ - scikit-learn hyperparameter**
Relative tolerance with regards to the Frobenius norm (Matrix Norm) of the difference in the cluster centers of two consecutive iterations to declate convergence.

Source: https://mathworld.wolfram.com/FrobeniusNorm.html

### Affinity Propagation

**Parameters** - Damping, Sample preference

**Scalability** - Not scalable with n_samples

**Usecase** - Many clusters, uneven cluster size, non-flat geometry

**Metric Used** - Graph distance (e.g. Nearest neighbor graph)

Amazing example: https://scikit-learn.org/stable/auto_examples/applications/plot_stock_market.html#sphx-glr-auto-examples-applications-plot-stock-market-py

Affinity Propagation creates clusters by sending messages between pairs of samples until convergence. A dataset is then described using a small number of exemplars, which are identified as those most representative of other samples. The messages sent between pairs represent the suitability for one sample to be the exemplar of the other, which is updated in response to the values from other pairs. This updating happens iteratively until convergence, at which point the final exemplars are chosen, and hence the final clustering is given.

Affinity Propagation chooses the number of clusters based on the data provided. For this purpose, the two most important parameters are the **preference**, which controls how many exemplars are used, and the **damping factor** which damps the responsibility and availability messages to avoid numerical oscillations when updating these messages.

The main drawback of Affinity Propagation is its complexity. The complexity is O($N^2T$), where N is the number of samples and T is the number of iterations until convergence. Further, the memory complexity is O($N^2$) if a dense similarity matrix is used, but reducible if a sparse similarity matrix is used. So, Affinity Propagation is most appropriate for small to medium sized datasets.

#### Algorithm

The messages sent between points belong to one of two categories. The first is the responsibility $r(i, k)$, which is the accumulated evidence that sample k should be the exemplar for sample i. The second is the availability $a(i, k)$ which is the accumulated evidence that sample i should choose sample k to be its exemplar, and considers the values for all other samples that k should be an exemplar. In this way, exemplars are chosen by samples if they are (1) simple enough to many samples and (2) chosen by many samples to be representative of themselves.

More formally, the responsibility of a sample k to be the exemplar of sample i is given by:

$r(i,k) \leftarrow s(i,k) - max[\ a(i,k^{'}) + s(i,k^{'}) \forall k^{'} \neq k ] \$

Where $s(i,k)$ is the similarity between samples $i$ and $k$. The availability of samples $k$ to be the exemplar of sample $i$ is given by:

$a(i,k) \leftarrow min [\ 0,r(k,k) + \underset{{i^{'} s.t i^{'} \notin {\ i,k}\}}}{\sum}  r(i^{'},k) \ ]$

To begin with, all values for $r$ and $a$ are set to zero, and the calculation of each iterates until convergence. As duscussed, in order to avoid numerical oscillations when updating the messages, the damping factor $\lambda$ is introduced to the iteration process:

$r_{t+1}(i,k) = \lambda \cdot r_t(i,k) + (1 - \lambda) \cdot r_{t+1}(i,k)$

$a_{t+1}(i,k) = \lambda \cdot a_t(i,k) + (1 - \lambda) \cdot a_{t+1}(i,k)$

where $t$ indicates the iteration we're on.


#### Final Notes

For the scikit-learn implementation we have the following:

* **damping: float, default=0.5** - Damping factor (between 0.5 and 1) is the extent to which the current value is maintained relative to incoming values (weighted 1 - damping). This is in order to avoid numerical oscillations when updating these values (messages).

* **preference : array-like of shape (n_samples,) or float, default=None** - Preferences for each point - points with larger values of preferences are more likely to be chosen as exemplars. The number of exemplars, i.e. of clusters, is influenced by the input preference value. If the preferences are not passed as arguments, they will be set to the median of the input similarities.

Extra info: https://towardsdatascience.com/unsupervised-machine-learning-affinity-propagation-algorithm-explained-d1fef85f22c8

Paper: Clustering by Passing Messages Between Data Points
Brendan J. Frey and Delbert Dueck.

### Mean-shift

**Parameters** - Bandwidth

**Scalability** - Not scalable with n_samples

**Usecase** - Many clusters, uneven cluster size, non-flat geometry

**Metric Used** - Distances between points

Mean-shift clustering aims to discover blobs in a smooth density of samples. it is a centroid based algorithm, which works by updating candidates for centroids to be the mean of the points within a given region. These candidates are then flitered in a post-processing stage to eliminate near-duplicates to form the final set of centroids. 

Mean-shift builds upn the concept of kernel density estimation.

Given a candidate $x_i$ for iteration $t$, the candidate is updated according to the following equation:

<p style="text-align: center;">
    $x_u^{t+1} = m(x_i^t)$
</p>

Given that $N(x_i)$ is the neighborhood of samples within a given distance around $x_i$ and $m$ is the mean shift vector that is computed for each centroid that points towards a region of the maximum increase in the density of points. This is computed using the following equation, effectively updating a centroid to be the mean of the samples within its neighborhood:

<p style="text-align: center;">
    $m(x_i) = \frac {\underset{x_j \in N(x_i)}{\sum} K(x_j - x_i)x_j}{\underset{x_j \in N(x_i)}{\sum} K(x_j - x_i)}$
</p>

Where K(...) is the kernel function.

The algorithm automatically sets the number of clusters, relying on the parameter bandwidth, which dictates the size of the region to search through. This parameter can be set (in sci-kit learn) or can be estimated which is called if the bandwidth is not set.

The algorithm is not highly scalable, as it requires multiple nearest neighbor searches during the execution of the algorithm. The algorithm is guaranteed to converge, however the algorithm will stop iteration when the change in centroids is small.

#### Kernel Density Estimation (KDE)

A **Kernel** is a probability density function $f(x)$ which is symmetric around the y axis.

**Kernel Density Estimation (KDE)** is a non-parametric method for estimating the pdf of a random variable based on a random sample using some kernel $K$ and some smoothing parameter (bandwidth) h > 0.

While a histogram counts the number of data points in arbitrary regions, a kernel density estimate is a function defined as the sum of a kernel function on every point. The Kernel function ($K(x)$) typically exhibits the following:

1. Symmetry such taht $K(u) = K(-u)$
2. Normalization such that $\int_{- \infty}^{\infty}$ K(u)du = 1
3. Monotonically decreasing such that $K'(u) < 0$ when $u > 0$
4. Expected value equal to zero such that $E[K] = 0$

Let $\{x_1,x_2,...,x_n \}$ be a random sample from some distribuution whose pdf $f(x)$ is not known. We estimate $f(x)$ with:

<p style="text-align: center;"> 
    $f(x) = \frac {1}{nh} \sum_{i=1}^{n} K(\frac {x - x_i}{h})$
</p>

The results are sensative to the value chosen for h. Rules for chosen an optimum value for h are complex, but the following are some simple guidelines:

* We should use a larger bandwidth value when the sample size is small and the data are sparse. This results in a larger standard deviation; the estimate places more weight on the neighboring data values

* We can use a smaller bandwidth value when the sample size is large and the data is densely packed. This results in a smaller standard devation; the estimate places more weight on the specific data value and less on the neighboring data values

Bandwidths that are too small results in a pdf that is too spiky, while bandiwdths that are too large results in a pdf that is over-smoothed. 

Sources:
https://www.real-statistics.com/distribution-fitting/kernel-density-estimation/

https://www.statsmodels.org/stable/examples/notebooks/generated/kernel_density.html

https://deepai.org/machine-learning-glossary-and-terms/kernel-density-estimation

https://en.wikipedia.org/wiki/Kernel_density_estimation

#### Radial Bias Function (RBF) Kernel

The sci-kit learn implementation of the Mean Shift algorithm uses the RBF kernel, so it's important to understand how it works.

RBF kernels are the most generalized formof kernelization and is one of the most widely used kernels due to its similarity to the Gaussian distribution. The RBF kernel function for two points $X_1$ and $X_2$ computes the similarity or how close they are to each other. This kernel can be represented as follows:

<p style="text-align: center;"> 
    $K(X_1,X_2) = K(X_1 - X_2) = exp(- \frac { \lVert X_1 - X_2 \rVert }{2 \sigma ^2})$
</p>

where,

1. $\sigma$ is the variance.
2. $ \lVert X_1 - X_2 \rVert$ is the Euclidean (L2-norm) Distance between $X_2$ and $X_2$

The maximum value that the RBF kernel can result in is 1 and occures when the distance is 0 which is when the points are the same.

1. When the points are the same, there is no distance between them and therefore they are extremely similar
2. When the points are separated by a large distance, then the kernel value is less than 1 and close to 0 which means the points are dissimilar

Source: https://towardsdatascience.com/radial-basis-function-rbf-kernel-the-go-to-kernel-acf0d22c798a < - Further explanation and examples!

#### Final Notes

The complexity of the Mean Shift algorithm will tend towards $O(T \cdot n \cdot log(n))$ in lower dimensions, with n the number of samples and T the number of points. In higher dimensions the complexity will tend towards $O(T \cdot n^2)$.

estimate_bandwidth (the method for which we can estimate the bandwidth when one is not set) is much less scalable than Mean Shift normally. This means it will often be the bottleneck if it's used.

* **seeds : array-like of shape (n_samples, n_features), default = None** - Seeds used to initialize kernels. If not set, the seeds are calculated by get_bin_seeds with bandwidth as the grid size and default values for other parameters. (Think bin size)

Scalability can be boosted by using fewer seeds.

Estimate bandwidth: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.estimate_bandwidth.html#sklearn.cluster.estimate_bandwidth

Source: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html#sklearn.cluster.MeanShift

Extra info (on Mean-Shift): https://www.geeksforgeeks.org/ml-mean-shift-clustering/

Extra info (on KDE): https://stackabuse.com/kernel-density-estimation-in-python-using-scikit-learn/

Paper: https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.76.8968&rep=rep1&type=pdf

### Spectral Clustering

**Parameters** - Number of Clusters

**Scalability** - Medium n_samples, small n_clusters

**Usecase** - Few cllusters, even cluster size, non-flat geometry

**Metric Used** - Graph distance (e.g. nearest neighbor graph)



**READ THE REFERENCE FIRST**

Reference: https://people.csail.mit.edu/dsontag/courses/ml14/notes/Luxburg07_tutorial_spectral_clustering.pdf